In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 14.7 MB/s 
     |████████████████████████████████| 7.6 MB 56.8 MB/s 
     |████████████████████████████████| 182 kB 24.6 MB/s 


In [2]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
import copy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_path = "/content/drive/MyDrive/Data/Thesis/Kaggle/filtered.csv"

df = pd.read_csv(data_path)

In [5]:
df.head()

,Unnamed: 0,Body,Label
0,0,naturally irresistible corporate identity lt r...,1
1,1,stock trading gunslinger fanny merrill muzo co...,1
2,2,unbelievable new homes made easy im wanting sh...,1
3,3,color printing special request additional in...,1
4,4,money get software cds software compatibility ...,1


In [6]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer):

        self.labels = [label for label in df['Label']]
        self.texts = [tokenizer(text, 
                                padding='max_length', 
                                max_length = 512, 
                                truncation=True,
                                return_tensors="pt") for text in df['Body']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [7]:
class BertClassifier(nn.Module):

    def __init__(self, bert, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = bert
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [18]:
class Client(object):
    def __init__(self, name, df_train, df_val, model, criterion, optimizer, tokenizer, epochs):
        self.name = name
        self.train_dataloader = torch.utils.data.DataLoader(Dataset(df_train, tokenizer), batch_size=2, shuffle=True)
        self.val_dataloader = torch.utils.data.DataLoader(Dataset(df_val, tokenizer), batch_size=2, shuffle=True)
        self.len_train = len(df_train)
        self.len_val = len(df_val)
        self.len_test = len(df_test)
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.tokenizer = tokenizer
        self.epochs = epochs

    def train(self, weights):
        self.model.load_state_dict(weights)    

        train_acc = []
        train_loss = []
        val_acc = []
        val_loss = []

        use_cuda = torch.cuda.is_available()
        device = torch.device("cuda" if use_cuda else "cpu")

        if use_cuda:
            self.model = self.model.cuda()
            self.criterion = self.criterion.cuda()
        
        for epoch_num in range(self.epochs):

            print(f"Client: {self.name}, Epoch: {epoch_num + 1}")

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(self.train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = self.model(input_id, mask)
                
                batch_loss = self.criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                self.model.zero_grad()
                batch_loss.backward()
                self.optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in self.val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = self.model(input_id, mask)

                    batch_loss = self.criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            
            train_acc.append(total_acc_train / self.len_train)
            train_loss.append(total_loss_train / self.len_train)
            val_acc.append(total_acc_val / self.len_val)
            val_loss.append(total_loss_val / self.len_val)

            print(
                f'| Train Loss: {total_loss_train / self.len_train: .3f} \
                \n| Train Accuracy: {total_acc_train / self.len_train: .3f} \
                \n| Val Loss: {total_loss_val / self.len_val: .3f} \
                \n| Val Accuracy: {total_acc_val / self.len_val: .3f}')
            
        avg_train_acc = sum(train_acc) / self.epochs
        avg_train_loss = sum(train_loss) / self.epochs
        avg_val_acc = sum(val_acc) / self.epochs
        avg_val_loss = sum(val_loss) / self.epochs

        weights = self.model.state_dict()

        return weights, avg_train_acc, avg_train_loss, avg_val_acc, avg_val_loss


In [22]:
class Server(object):
    def __init__(self, df_train_splits, df_val_splits, df_test, model, criterion, optimizer, tokenizer, epochs, rounds, n_clients):
        self.df_train_splits = df_train_splits
        self.df_val_splits = df_val_splits
        self.df_test = df_test
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.tokenizer = tokenizer
        self.epochs = epochs
        self.rounds = rounds
        self.n_clients = n_clients

    def train(self):
        global_weights = self.model.state_dict()

        train_acc = {}
        train_loss = {}
        val_acc = {}
        val_loss = {}

        for i in range(self.n_clients):
            train_acc[f"client_{i + 1}"] = []
            train_loss[f"client_{i + 1}"] = []
            val_acc[f"client_{i + 1}"] = []
            val_loss[f"client_{i + 1}"] = []
        
        for curr_round in range(1, self.rounds+1):
            print(f"\n\nRound {curr_round}...")

            curr_weights = self.model.state_dict()

            w = []
            for i in range(self.n_clients):
                name = f"client_{i + 1}"
                client_i = Client(name, 
                                self.df_train_splits[i],
                                self.df_val_splits[i], 
                                self.model, 
                                self.criterion, 
                                self.optimizer, 
                                self.tokenizer, 
                                self.epochs)

                weights, avg_train_acc, avg_train_loss, avg_val_acc, avg_val_loss = client_i.train(curr_weights)

                w.append(copy.deepcopy(weights))

                train_acc[f"client_{i + 1}"].append(copy.deepcopy(avg_train_acc))
                train_loss[f"client_{i + 1}"].append(copy.deepcopy(avg_train_loss))
                val_acc[f"client_{i + 1}"].append(copy.deepcopy(avg_val_acc))
                val_loss[f"client_{i + 1}"].append(copy.deepcopy(avg_val_loss))
            
            weights_avg = copy.deepcopy(w[0])

            for k in weights_avg.keys():
                for i in range(1, len(w)):
                    weights_avg[k] += w[i][k]
                weights_avg[k] = torch.div(weights_avg[k], len(w))
            
            global_weights = weights_avg

            self.model.load_state_dict(global_weights)
        
        print(train_acc, train_loss, val_acc, val_loss)
    
    def test(self):
        test_dataloader = torch.utils.data.DataLoader(Dataset(self.df_test, self.tokenizer), batch_size=2)
        len_test = len(self.df_test)
        use_cuda = torch.cuda.is_available()
        device = torch.device("cuda" if use_cuda else "cpu")

        if use_cuda:
            self.model = self.model.cuda()

        total_acc_test = 0
        with torch.no_grad():
            for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = self.model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
        print(f'Test Accuracy: {total_acc_test / len_test: .3f}')


In [10]:
def split_dataframe(df, n):
    s = int(len(df) / n)
    shuffled = df.sample(frac = 1)
    splits = []

    for i in range(n):
        if i != n - 1:
            splits.append(shuffled.iloc[i*s:(i+1)*s])
        else:
            splits.append(shuffled.iloc[i*s:])
    
    return splits

In [11]:
n_clients = 8
rounds = 2

In [12]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

4582 573 573


In [13]:
df_train_splits = split_dataframe(df_train, n_clients)
df_val_splits = split_dataframe(df_val, n_clients)

In [14]:
epochs = 4
learning_rate = 1e-6

bert = BertModel.from_pretrained('bert-base-cased')
model = BertClassifier(bert)

criterion = nn.CrossEntropyLoss()

optimizer = Adam(model.parameters(), lr= learning_rate)

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [23]:
server = Server(df_train_splits, df_val_splits, df_test, model, criterion, optimizer, tokenizer, epochs, rounds, n_clients)

In [16]:
server.train()



Round 1...
Client: client_1, Epoch: 1


100%|██████████| 286/286 [01:02<00:00,  4.61it/s]


| Train Loss:  0.345                 
| Train Accuracy:  0.680                 
| Val Loss:  0.318                 
| Val Accuracy:  0.803
Client: client_1, Epoch: 2


100%|██████████| 286/286 [01:00<00:00,  4.75it/s]


| Train Loss:  0.305                 
| Train Accuracy:  0.743                 
| Val Loss:  0.228                 
| Val Accuracy:  0.845
Client: client_1, Epoch: 3


...

| Train Loss:  0.011                 
| Train Accuracy:  0.991                 
| Val Loss:  0.005                 
| Val Accuracy:  1.000
Client: client_8, Epoch: 3


100%|██████████| 289/289 [01:01<00:00,  4.73it/s]


| Train Loss:  0.005                 
| Train Accuracy:  0.998                 
| Val Loss:  0.004                 
| Val Accuracy:  1.000
Client: client_8, Epoch: 4


100%|██████████| 289/289 [01:01<00:00,  4.72it/s]


| Train Loss:  0.005                 
| Train Accuracy:  0.998                 
| Val Loss:  0.003                 
| Val Accuracy:  1.000
{'client_1': [0.8024475524475525, 0.9121503496503496], 'client_2': [0.6700174825174825, 0.9763986013986015], 'client_3': [0.7320804195804196, 0.9881993006993006], 'client_4': [0.7465034965034965, 0.99256993006993], 'client_5': [0.7027972027972028, 0.9886363636363638], 'client_6': [0.7604895104895104, 0.9921328671328671], 'client_7': [0.6975524475524475, 0.9895104895104895], 'client_8': [0.7192906574394463, 0.9891868512110726]} {'client_1': [0.2432154927832576, 0.12135735973088929], 'client_2': [0.3443219023333354, 0.053656871993521274], 'client_3': [0.3347819337452, 0.02812543560643322], 'client_4': [0.30022752062919045, 0.020442941820556688], 'client_5': [0.34691522973817546, 0.024596316207194595], 'client_6': [0.2799864777484322, 0.018266935363473773], 'client_7': [0.3328578777975969, 0.017751491370672355], 'client_8': [0.3474844759362379, 0.01471

In [24]:
server.test()

Test Accuracy:  0.984
